In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np
from sentence_transformers import SentenceTransformer

In [2]:

train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("val.csv")
test_df = pd.read_csv("test.csv")

In [3]:
model = SentenceTransformer("all-MiniLM-L6-v2")
X_train = model.encode(train_df["query"].tolist(), convert_to_tensor=True)
X_val = model.encode(val_df["query"].tolist(), convert_to_tensor=True)
X_test = model.encode(test_df["query"].tolist(), convert_to_tensor=True)

X_train = X_train.unsqueeze(1)
X_val   = X_val.unsqueeze(1)
X_test  = X_test.unsqueeze(1)

y_train = torch.tensor(train_df["carb"].values, dtype=torch.float32).unsqueeze(1)
y_val   = torch.tensor(val_df["carb"].values, dtype=torch.float32).unsqueeze(1)
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
val_loader   = DataLoader(TensorDataset(X_val, y_val), batch_size=32)

X_train.shape

torch.Size([8000, 1, 384])

In [4]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()

    def forward(self, yhat, y):
        return torch.sqrt(self.mse(yhat, y))

In [50]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size=384, hidden_size=128, num_layers=3, dropout=0.2, bidirectional=False):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(
            input_size=input_size, 
            hidden_size=hidden_size, 
            num_layers=num_layers, 
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0, 
            bidirectional=bidirectional, 
            nonlinearity='relu'
            )
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.rnn(x)
        return self.fc(out[:, -1, :])    


In [51]:
model = RNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
criterion = nn.MSELoss()

epochs = 100

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        preds = model(x_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    rmse = avg_loss ** 0.5
    print(f"Epoch {epoch+1}, Train MSE: {avg_loss:.4f}, RMSE: {rmse:.2f} grams")

    

Epoch 1, Train MSE: 1668.2448, RMSE: 40.84 grams
Epoch 2, Train MSE: 1312.0507, RMSE: 36.22 grams
Epoch 3, Train MSE: 1176.1080, RMSE: 34.29 grams
Epoch 4, Train MSE: 1047.4256, RMSE: 32.36 grams
Epoch 5, Train MSE: 939.4315, RMSE: 30.65 grams
Epoch 6, Train MSE: 818.0128, RMSE: 28.60 grams
Epoch 7, Train MSE: 737.5461, RMSE: 27.16 grams
Epoch 8, Train MSE: 676.6195, RMSE: 26.01 grams
Epoch 9, Train MSE: 604.9342, RMSE: 24.60 grams
Epoch 10, Train MSE: 541.7374, RMSE: 23.28 grams
Epoch 11, Train MSE: 471.5642, RMSE: 21.72 grams
Epoch 12, Train MSE: 447.6483, RMSE: 21.16 grams
Epoch 13, Train MSE: 420.2050, RMSE: 20.50 grams
Epoch 14, Train MSE: 383.8927, RMSE: 19.59 grams
Epoch 15, Train MSE: 378.7500, RMSE: 19.46 grams
Epoch 16, Train MSE: 341.0446, RMSE: 18.47 grams
Epoch 17, Train MSE: 364.5363, RMSE: 19.09 grams
Epoch 18, Train MSE: 356.9507, RMSE: 18.89 grams
Epoch 19, Train MSE: 318.3595, RMSE: 17.84 grams
Epoch 20, Train MSE: 298.3174, RMSE: 17.27 grams
Epoch 21, Train MSE: 268.

In [53]:
model.eval()
total_val_loss = 0
all_preds = []
all_targets = []

with torch.no_grad():
    for x_batch, y_batch in val_loader:

        preds = model(x_batch)
        loss = criterion(preds, y_batch)
        total_val_loss += loss.item()

        all_preds.append(preds.cpu())
        all_targets.append(y_batch.cpu())

avg_val_loss = total_val_loss / len(val_loader)
rmse = avg_val_loss ** 0.5
print(f"Validation MSE: {avg_val_loss:.4f}, RMSE: {rmse:.2f} grams")

Validation MSE: 391.6109, RMSE: 19.79 grams


In [54]:
model.eval()
with torch.no_grad():
    preds = model(X_val).squeeze().numpy()

# Add prediction column and save
test_df["carb"] = preds
test_df.to_csv("test_with_predictions_transformer_rnn.csv", index=False)